### Importing Libraries

In [2]:
import zipfile, sys, warnings, cv2
from math import ceil
import numpy as np
import scipy.misc as sm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import figure
from termcolor import colored
from keras.preprocessing import image
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, LearningRateScheduler
from keras import backend as K
from keras.models import load_model

Using TensorFlow backend.


#### Set Default Configuration Some of the Package

In [3]:
%pylab inline
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['ceil']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


#### Load the Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### This will Extract All the Files to the Current Working Directory. 

In [5]:
proj_path = "/content/drive/My Drive/Colab Notebooks/DLCP/FaceDetection-MobileNet/"
zip_file = "Files_required_for_face_detection.zip"

#This Extracts the Files to a New Directory Named MobileNet
archive = zipfile.ZipFile(proj_path + zip_file, 'r')
archive.extractall(proj_path + 'MobileNet/')

# Set the Extracted Folder to sys.path to be able to load the mn_model
sys.path.append(proj_path + 'MobileNet/')

%cd $proj_path
%cd 'MobileNet/'

/content/drive/My Drive/Colab Notebooks/DLCP/FaceDetection-MobileNet
/content/drive/My Drive/Colab Notebooks/DLCP/FaceDetection-MobileNet/MobileNet


#### Import MobileNet Model

In [0]:
from mn_model import mn_model

#### Import the BatchGenerator and SSD Loss Functions

In [0]:
from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

### Set the parameters for Customize the Model Parameters and Set class_names = ["background", "face"]

In [0]:
img_height =512
img_width = 512

img_channels = 3

n_classes =2 
class_names = ["background","face"]

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # anchorboxes for coco dataset
aspect_ratios = [[0.5, 1.0, 2.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [0.5, 1.0, 2.0],
                 [0.5, 1.0, 2.0]] 
two_boxes_for_ar1 = True
limit_boxes = True 
variances = [0.1, 0.1, 0.2, 0.2] 
coords = 'centroids' 
normalize_coords = True

det_model_path = "weights/"

In [0]:
train_images_path = proj_path + 'WIDER_train.zip'
test_images_path = proj_path + 'WIDER_val.zip'

#### Extracts the Train Images to the Current Working Directory

In [0]:
archive = zipfile.ZipFile(train_images_path, 'r')
archive.extractall()

#### Extracts the Valdation Images to the Current Working Directory

In [0]:
archive = zipfile.ZipFile(test_images_path, 'r')
archive.extractall()

### Model Using Transfer Learning

In [0]:
data = np.load('./wider_train_small.npy', allow_pickle=True).item()

In [13]:
for key in data:
    print(key)
    print(data[key])
    break

52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg
['WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', 'WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', [300, 300], [[21, 667, 9, 655], 1]]


### Load Two Files `wider_trian_v1.npy` & `wider_val_v1.npy`

In [0]:
train_data = 'wider_train_small.npy'
val_data = 'wider_val_small.npy'

#### The "model" is the Keras Model Has to be Trained 

In [15]:
K.clear_session()
model, model_layer, img_input, predictor_sizes = mn_model(image_size=(img_height, img_width, img_channels), 
                                                                      n_classes = n_classes,
                                                                      min_scale = None, 
                                                                      max_scale = None, 
                                                                      scales = scales, 
                                                                      aspect_ratios_global = None, 
                                                                      aspect_ratios_per_layer = aspect_ratios, 
                                                                      two_boxes_for_ar1= two_boxes_for_ar1, 
                                                                      limit_boxes=limit_boxes, 
                                                                      variances= variances, 
                                                                      coords=coords, 
                                                                      normalize_coords=normalize_coords)

model.summary()




====> Model Specific data
====> Height, Width, Channels : 512 512 3









__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda1 (Lambda)                (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda2 (Lambda)                (None, 512, 512, 3)  0           lambda1[0][0]                    
__________________________________________________________________________________________________
lambda3 (Lambda)                (None, 512, 512, 3)  0           lambda2[0][0]                    
_____________________________

#### Freeze Classification Layers

In [16]:
for layer in model_layer:
  model_layer[layer].trainable = layer.startswith('detection')
print(colored(f'Total {len([layer for layer in model_layer if model_layer[layer].trainable == False])} layers are freezed', 'green'))

Total 85 layers are freezed


### Transfer Learning is Load the weights of the Model Given in the File 

In [17]:
trained_weights = 'mobilenet_1_0_224_tf.h5'
model.load_weights(trained_weights, by_name=True)
print(colored('Weights loaded successfully', 'green'))

Weights loaded successfully


#### Using the Functions the Model, Split the Dataset into Train and Valid.

In [18]:
batch_size = 32
ssd_box_encoder = SSDBoxEncoder(img_height=img_height,
                                img_width=img_width,
                                n_classes=n_classes, 
                                predictor_sizes=predictor_sizes,
                                min_scale=None,
                                max_scale=None,
                                scales=scales,
                                aspect_ratios_global=None,
                                aspect_ratios_per_layer=aspect_ratios,
                                two_boxes_for_ar1=two_boxes_for_ar1,
                                limit_boxes=limit_boxes,
                                variances=variances,
                                pos_iou_threshold=0.5,
                                neg_iou_threshold=0.2,
                                coords=coords,
                                normalize_coords=normalize_coords)

train_dataset = BatchGenerator(images_path=train_data, 
                include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("==>TRAINING DATA")
print ("==> Parsing XML files ...")

train_dataset.parse_xml(
                  annotations_path=train_data,
                  image_set_path='None',
                  image_set='None',
                  classes = class_names, 
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)
print("==>Parsing XML Finished.")

print ("==>Generate training batches...")
train_generator = train_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=True,
                 brightness=(0.5,2,0.5),
                 flip=0.5,
                 translate=((0, 20), (0, 30), 0.5),
                 scale=(0.75, 1.2, 0.5),
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False,
                 resize=(img_height, img_width),
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

print ("==>Training batch generation complete")

n_train_samples = train_dataset.get_n_samples()

print ("==>Total number of training samples = {}".format(n_train_samples))

# Now repeat above steps for validation data 

print ("\n==>VALIDATION")

val_dataset = BatchGenerator(images_path=val_data, include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("==> Parsing XML files ...")


val_dataset.parse_xml(
                  annotations_path=val_data,
                  image_set_path='None',
                  image_set='None',
                  classes = class_names, 
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)


print("==>Parsing XML Finished.")


print ("==>Generate validation batches...")
val_generator = val_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False, 
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)


print ("==>Validation batch generation complete")

n_val_samples = val_dataset.get_n_samples()

print ("==>Total number of validation samples = {}".format(n_val_samples))

==>TRAINING DATA
==> Parsing XML files ...
==>Parsing XML Finished.
==>Generate training batches...
==>Training batch generation complete
==>Total number of training samples = 128

==>VALIDATION
==> Parsing XML files ...
==>Parsing XML Finished.
==>Generate validation batches...
==>Validation batch generation complete
==>Total number of validation samples = 60


### Learning Rate, Epochs, Optimizer & Loss Function (SSD Loss) to Compile the Model

In [19]:
batch_size = 16
num_epochs = 30

#Learning rate
base_lr = 0.002

# Optimizer
adam = Adam(lr=base_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-6, decay = 0.0)

# Loss
ssd_loss = SSDLoss(neg_pos_ratio=2, n_neg_min=0, alpha=1.0, beta = 1.0)

# Compile
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)



Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.


#### Layers Using model.fit_generator & Train the Model

In [20]:
# Define the Callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=100)
model_checkpoint = ModelCheckpoint(det_model_path + 'MobileNet_FaceDetection.Model_epoch{epoch:02d}_loss{val_loss:.4f}.h5',
                                                           monitor='val_loss',
                                                           verbose=1,
                                                           save_best_only=True,
                                                           save_weights_only=True,
                                                           mode='auto',
                                                           period=1)

# Train & Validate the Model & Save the Best Weights
print("Training on WIDER face data...")
model.fit_generator(generator = train_generator,
                              steps_per_epoch = ceil(n_train_samples/batch_size)*2,
                              epochs = num_epochs,
                              callbacks = [model_checkpoint, early_stopping],                      
                              validation_data = val_generator,
                              validation_steps = ceil(n_val_samples/batch_size))

model.save_weights(det_model_path + 'MobileNet_FaceDetection.Weights_epoch{}.h5'.format(num_epochs))

print(colored("Model and best weights are saved at: " + det_model_path, 'green'))

Training on WIDER face data...


Epoch 1/30
16/16 [==============================] - 54s 3s/step - loss: 0.3209 - val_loss: 0.3243

Epoch 00001: val_loss improved from inf to 0.32429, saving model to weights/MobileNet_FaceDetection.Model_epoch01_loss0.3243.h5
Epoch 2/30
16/16 [==============================] - 15s 931ms/step - loss: 0.2485 - val_loss: 0.2776

Epoch 00002: val_loss improved from 0.32429 to 0.27758, saving model to weights/MobileNet_FaceDetection.Model_epoch02_loss0.2776.h5
Epoch 3/30
16/16 [==============================] - 34s 2s/step - loss: 0.2294 - val_loss: 0.2523

Epoch 00003: val_loss improved from 0.27758 to 0.25233, saving model to weights/MobileNet_FaceDetection.Model_epoch03_loss0.2523.h5
Epoch 4/30
16/16 [==============================] - 35s 2s/step - loss: 0.2180 - val_loss: 0.2399

Epoch 00004: val_loss improved from 0.25233 to 0.23994, saving model to weights/MobileNet_FaceDetection.Model_epoch04_loss0.2399.h5
Epoch 5/30
16/16 [==========================

### Function to Plot the Bounding Box in Test Image to Show the Predictions

In [0]:
def save_bb(path, filename, results, prediction=True):
  
  # print FileName

  img = image.load_img(filename, target_size=(img_height, img_width))
  img = image.img_to_array(img)

  filename = filename.split("/")[-1]

  if(not prediction):
    filename = filename[:-4] + "_gt" + ".jpg"

  #fig,currentAxis = plt.subplots(1)
  currentAxis = plt.gca()

 # Get Detections With Confidence Higher Than 0.6.
  colors = plt.cm.hsv(np.linspace(0, 1, len(results))).tolist()
  color_code = len(results)
  print (colored("total number of bbs: %d" % len(results), "yellow"))
  for result in results:
    # Parse the Outputs.

    if(prediction):
      det_label = result[0]
      det_conf = result[1]
      det_xmin = result[2]
      det_xmax = result[3]
      det_ymin = result[4]
      det_ymax = result[5]
    else :
      det_label = result[0]
      det_xmin = result[1]
      det_xmax = result[2]
      det_ymin = result[3]
      det_ymax = result[4]

    xmin = int(det_xmin)
    ymin = int(det_ymin)
    xmax = int(det_xmax)
    ymax = int(det_ymax)

    if(prediction):
      score = det_conf
    
    plt.imshow(img / 255.)
    
    label = int(int(det_label))
    label_name = class_names[label]
    # print label_name 
    # print label

    if(prediction):
      display_txt = '{:0.2f}'.format(score)
    else:
      display_txt = '{}'.format(label_name)

      
    # print (xmin, ymin, ymin, ymax)
    coords = (xmin, ymin), (xmax-xmin), (ymax-ymin)
    color_code = color_code-1 
    color = colors[color_code]
    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
    currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.2})

  # y
  currentAxis.axes.get_yaxis().set_visible(False)
  # x
  currentAxis.axes.get_xaxis().set_visible(False)
  plt.savefig(path + filename, bbox_inches='tight')

  print ('saved' , path + filename)

  plt.clf()

In [24]:
val_size = 16
test_generator = val_dataset.generate(
                 batch_size=val_size,
                 train=False,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

print (colored("done.", "green"))

print (colored("now predicting...", "yellow"))

_CONF = 0.60 
_IOU = 0.15

for j in range(val_size):
  X, y, filenames = next(test_generator)
  num_files = len(filenames)

  y_pred = model.predict(X)


  y_pred_decoded = decode_y2(y_pred,
                            confidence_thresh=_CONF,
                            iou_threshold=_IOU,
                            top_k='all',
                            input_coords=coords,
                            normalize_coords=normalize_coords,
                            img_height=img_height,
                            img_width=img_width)


  np.set_printoptions(suppress=True)
  
  for i in range(num_files):
    save_bb("./output_test/", filenames[i], y_pred_decoded[i])
    save_bb("./output_test/", filenames[i], y[i], prediction=False)

done.
now predicting...
total number of bbs: 5
saved ./output_test/2_Demonstration_Protesters_2_646.jpg
total number of bbs: 14
saved ./output_test/2_Demonstration_Protesters_2_646_gt.jpg
total number of bbs: 3
saved ./output_test/2_Demonstration_Demonstrators_2_188.jpg
total number of bbs: 12
saved ./output_test/2_Demonstration_Demonstrators_2_188_gt.jpg
total number of bbs: 1
saved ./output_test/33_Running_Running_33_209.jpg
total number of bbs: 1
saved ./output_test/33_Running_Running_33_209_gt.jpg
total number of bbs: 1
saved ./output_test/39_Ice_Skating_Ice_Skating_39_270.jpg
total number of bbs: 1
saved ./output_test/39_Ice_Skating_Ice_Skating_39_270_gt.jpg
total number of bbs: 0
saved ./output_test/33_Running_Running_33_475.jpg
total number of bbs: 1
saved ./output_test/33_Running_Running_33_475_gt.jpg
total number of bbs: 6
saved ./output_test/12_Group_Large_Group_12_Group_Large_Group_12_448.jpg
total number of bbs: 5
saved ./output_test/12_Group_Large_Group_12_Group_Large_Grou

## OBSERVATIONS:

##### - Importing Libraries
##### - Set Default Configuration Some of the Package
##### - Load the Google Drive
##### - This will Extract All the Files to the Current Working Directory.
##### - Import MobileNet Model
##### - Import the BatchGenerator and SSD Loss Functions
##### - Set the parameters for Customize the Model Parameters and Set class_names = ["background", "face"]
##### - Extracts the Train Images to the Current Working Directory
##### - Extracts the Valdation Images to the Current Working Directory
##### - Model Using Transfer Learning
##### - Load Two Files wider_trian_v1.npy & wider_val_v1.npy
##### - The "model" is the Keras Model Has to be Trained
##### - Freeze Classification Layers
##### - Transfer Learning is Load the weights of the Model Given in File
##### - Using the Functions the Model, Split the Dataset into Train and Valid
##### - Learning Rate, Epochs, Optimizer & Loss Function (SSD Loss) to Compile the Model
##### - Layers Using model.fit_generator & Train the Model
##### - Function to Plot the Bounding Box in Test Image to Show the Predictions
##### - Get detections with confidence higher than 0.6.
    
    
    
    
    